In [46]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import requests
from bs4 import BeautifulSoup
!pip install geocoder
import geocoder
from geopy.geocoders import Nominatim
import folium
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans

print("All libraries installed!!")

All libraries installed!!


# Fetching Data

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
r = requests.get(url) 
  
soup = BeautifulSoup(r.content, 'lxml')
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"XptG7wpAMNAAAUaw2@sAAAES","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":951325562,"wgRevisionId":951325562,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Communications in Ontario","Postal codes in Canada","Toronto","Ontario

In [3]:
table = soup.find_all('td')
print(table)

[<td>M1A
</td>, <td>Not assigned
</td>, <td>
</td>, <td>M2A
</td>, <td>Not assigned
</td>, <td>
</td>, <td>M3A
</td>, <td>North York
</td>, <td>Parkwoods
</td>, <td>M4A
</td>, <td>North York
</td>, <td>Victoria Village
</td>, <td>M5A
</td>, <td>Downtown Toronto
</td>, <td>Regent Park / Harbourfront
</td>, <td>M6A
</td>, <td>North York
</td>, <td>Lawrence Manor / Lawrence Heights
</td>, <td>M7A
</td>, <td>Downtown Toronto
</td>, <td>Queen's Park / Ontario Provincial Government
</td>, <td>M8A
</td>, <td>Not assigned
</td>, <td>
</td>, <td>M9A
</td>, <td>Etobicoke
</td>, <td>Islington Avenue
</td>, <td>M1B
</td>, <td>Scarborough
</td>, <td>Malvern / Rouge
</td>, <td>M2B
</td>, <td>Not assigned
</td>, <td>
</td>, <td>M3B
</td>, <td>North York
</td>, <td>Don Mills
</td>, <td>M4B
</td>, <td>East York
</td>, <td>Parkview Hill / Woodbine Gardens
</td>, <td>M5B
</td>, <td>Downtown Toronto
</td>, <td>Garden District, Ryerson
</td>, <td>M6B
</td>, <td>North York
</td>, <td>Glencairn
</td>, <td>M7

# Parsing Data

In [4]:
postalCodes = [];
boroughs= [];
neighborhoods = [];
columnNum = 1;
passVal = False

for row in soup.find_all('td'):
    for cell in row:
        if cell.string and cell.string[0].isalpha() and len(cell.string) > 2:
            passVal = False
            if columnNum == 1:
                if passVal == False and cell.string[1].isdigit():
                    postalCodes.append(cell.string);   
                    columnNum = 2
                else:
                    continue
            elif columnNum == 2 :
                if cell.string == 'Not assigned':
                    passVal = True
                    del postalCodes[-1]
                    columnNum = 1
                    continue
                else:
                    boroughs.append(cell.string);      
                    columnNum = 3
            elif columnNum == 3 :
                if cell.string == 'Not assigned\n':
                    neighborhoods.append(boroughs[-1])
                else:
                    neighborhoods.append(cell.string); 
                columnNum = 1
                
print('Data Collected.')

Data Collected.


In [5]:
print(postalCodes)

['M1A\n', 'M3A\n', 'M4A\n', 'M5A\n', 'M6A\n', 'M7A\n', 'M8A\n', 'M1B\n', 'M2B\n', 'M4B\n', 'M5B\n', 'M6B\n', 'M7B\n', 'M9B\n', 'M1C\n', 'M2C\n', 'M4C\n', 'M5C\n', 'M6C\n', 'M7C\n', 'M9C\n', 'M1E\n', 'M2E\n', 'M4E\n', 'M5E\n', 'M6E\n', 'M7E\n', 'M9E\n', 'M2G\n', 'M4G\n', 'M5G\n', 'M6G\n', 'M7G\n', 'M9G\n', 'M2H\n', 'M3H\n', 'M4H\n', 'M5H\n', 'M6H\n', 'M7H\n', 'M9H\n', 'M2J\n', 'M3J\n', 'M4J\n', 'M5J\n', 'M6J\n', 'M7J\n', 'M9J\n', 'M2K\n', 'M3K\n', 'M4K\n', 'M5K\n', 'M6K\n', 'M7K\n', 'M9K\n', 'M2L\n', 'M3L\n', 'M4L\n', 'M5L\n', 'M6L\n', 'M7L\n', 'M9L\n', 'M1M\n', 'M2M\n', 'M3M\n', 'M4M\n', 'M5M\n', 'M6M\n', 'M7M\n', 'M9M\n', 'M1N\n', 'M2N\n', 'M3N\n', 'M4N\n', 'M5N\n', 'M6N\n', 'M7N\n', 'M9N\n', 'M1P\n', 'M2P\n', 'M3P\n', 'M5P\n', 'M6P\n', 'M7P\n', 'M9P\n', 'M1R\n', 'M2R\n', 'M3R\n', 'M5R\n', 'M6R\n', 'M7R\n', 'M8R\n', 'M1S\n', 'M2S\n', 'M4S\n', 'M5S\n', 'M6S\n', 'M7S\n', 'M9S\n', 'M2T\n', 'M4T\n', 'M5T\n', 'M6T\n', 'M8T\n', 'M1V\n', 'M2V\n', 'M4V\n', 'M5V\n', 'M6V\n', 'M8V\n', 'M9V\n', 

In [6]:
print(boroughs)

['Not assigned\n', 'North York\n', 'North York\n', 'Downtown Toronto\n', 'North York\n', 'Downtown Toronto\n', 'Not assigned\n', 'Scarborough\n', 'Not assigned\n', 'East York\n', 'Downtown Toronto\n', 'North York\n', 'Not assigned\n', 'Etobicoke\n', 'Scarborough\n', 'Not assigned\n', 'East York\n', 'Downtown Toronto\n', 'York\n', 'Not assigned\n', 'Etobicoke\n', 'Scarborough\n', 'Not assigned\n', 'East Toronto\n', 'Downtown Toronto\n', 'York\n', 'Not assigned\n', 'Not assigned\n', 'Not assigned\n', 'East York\n', 'Downtown Toronto\n', 'Downtown Toronto\n', 'Not assigned\n', 'Not assigned\n', 'North York\n', 'North York\n', 'East York\n', 'Downtown Toronto\n', 'West Toronto\n', 'Not assigned\n', 'Not assigned\n', 'North York\n', 'North York\n', 'East York\n', 'Downtown Toronto\n', 'West Toronto\n', 'Not assigned\n', 'Not assigned\n', 'North York\n', 'North York\n', 'East Toronto\n', 'Downtown Toronto\n', 'West Toronto\n', 'Not assigned\n', 'Not assigned\n', 'North York\n', 'North York\n

In [7]:
print(neighborhoods)

['M2A\n', 'Parkwoods\n', 'Victoria Village\n', 'Regent Park / Harbourfront\n', 'Lawrence Manor / Lawrence Heights\n', "Queen's Park / Ontario Provincial Government\n", 'M9A\n', 'Malvern / Rouge\n', 'M3B\n', 'Parkview Hill / Woodbine Gardens\n', 'Garden District, Ryerson\n', 'Glencairn\n', 'M8B\n', 'West Deane Park / Princess Gardens / Martin Grove / Islington / Cloverdale\n', 'Rouge Hill / Port Union / Highland Creek\n', 'M3C\n', 'Woodbine Heights\n', 'St. James Town\n', 'Humewood-Cedarvale\n', 'M8C\n', 'Eringate / Bloordale Gardens / Old Burnhamthorpe / Markland Wood\n', 'Guildwood / Morningside / West Hill\n', 'M3E\n', 'The Beaches\n', 'Berczy Park\n', 'Caledonia-Fairbanks\n', 'M8E\n', 'M1G\n', 'M3G\n', 'Leaside\n', 'Central Bay Street\n', 'Christie\n', 'M8G\n', 'M1H\n', 'Hillcrest Village\n', 'Bathurst Manor / Wilson Heights / Downsview North\n', 'Thorncliffe Park\n', 'Richmond / Adelaide / King\n', 'Dufferin / Dovercourt Village\n', 'M8H\n', 'M1J\n', 'Fairview / Henry Farm / Oriole

In [8]:
postal_codes = [x.replace('\n', '') for x in postalCodes]
print(postal_codes)

['M1A', 'M3A', 'M4A', 'M5A', 'M6A', 'M7A', 'M8A', 'M1B', 'M2B', 'M4B', 'M5B', 'M6B', 'M7B', 'M9B', 'M1C', 'M2C', 'M4C', 'M5C', 'M6C', 'M7C', 'M9C', 'M1E', 'M2E', 'M4E', 'M5E', 'M6E', 'M7E', 'M9E', 'M2G', 'M4G', 'M5G', 'M6G', 'M7G', 'M9G', 'M2H', 'M3H', 'M4H', 'M5H', 'M6H', 'M7H', 'M9H', 'M2J', 'M3J', 'M4J', 'M5J', 'M6J', 'M7J', 'M9J', 'M2K', 'M3K', 'M4K', 'M5K', 'M6K', 'M7K', 'M9K', 'M2L', 'M3L', 'M4L', 'M5L', 'M6L', 'M7L', 'M9L', 'M1M', 'M2M', 'M3M', 'M4M', 'M5M', 'M6M', 'M7M', 'M9M', 'M1N', 'M2N', 'M3N', 'M4N', 'M5N', 'M6N', 'M7N', 'M9N', 'M1P', 'M2P', 'M3P', 'M5P', 'M6P', 'M7P', 'M9P', 'M1R', 'M2R', 'M3R', 'M5R', 'M6R', 'M7R', 'M8R', 'M1S', 'M2S', 'M4S', 'M5S', 'M6S', 'M7S', 'M9S', 'M2T', 'M4T', 'M5T', 'M6T', 'M8T', 'M1V', 'M2V', 'M4V', 'M5V', 'M6V', 'M8V', 'M9V', 'M1W', 'M2W', 'M4W', 'M5W', 'M6W', 'M8W', 'M9W', 'M1X', 'M2X', 'M4X', 'M5X', 'M6X', 'M8X', 'M9X', 'M2Y', 'M4Y', 'M5Y', 'M7Y', 'M8Y', 'M9Y', 'M2Z', 'M4Z', 'M6Z', 'M8Z', 'M9Z']


In [9]:
boroughs = [x.replace('\n', '') for x in boroughs]
neighborhoods = [x.replace('\n', '') for x in neighborhoods]

In [10]:
neighborhoods = [x.replace('/', ',') for x in neighborhoods]

In [11]:
# define the dataframe columns
column_names = ['PostalCode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighbors = pd.DataFrame(columns=column_names)

neighbors

,PostalCode,Borough,Neighborhood,Latitude,Longitude


In [12]:
# initialize your variable to None
lat_lng_coords = None

for data in range(len(neighborhoods)):
    code = postal_codes[data]
    borough = boroughs[data]
    neighborhood_name = neighborhoods[data]
    g = geocoder.arcgis('{}, Toronto, Ontario'.format(code))
    lat_lng_coords = g.latlng

    neighbors = neighbors.append({ 'PostalCode': code,
                                   'Borough': borough,
                                   'Neighborhood': neighborhood_name,
                                   'Latitude': lat_lng_coords[0],
                                   'Longitude': lat_lng_coords[1]}, ignore_index=True)

In [13]:
neighbors.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1A,Not assigned,M2A,43.648690,-79.385440
1,M3A,North York,Parkwoods,43.752935,-79.335641
2,M4A,North York,Victoria Village,43.728102,-79.311890
3,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.650964,-79.353041
4,M6A,North York,"Lawrence Manor , Lawrence Heights",43.723265,-79.451211


In [14]:
neighbors.shape

(135, 5)

## Using Geopy to get coordinates of Toronto

In [15]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="on_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.6534817, -79.3839347.


In [16]:
# create map of New York using latitude and longitude values
map_on = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighbors['Latitude'], neighbors['Longitude'], neighbors['Borough'], neighbors['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_on)  
    
map_on

In [17]:
neighbors.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1A,Not assigned,M2A,43.648690,-79.385440
1,M3A,North York,Parkwoods,43.752935,-79.335641
2,M4A,North York,Victoria Village,43.728102,-79.311890
3,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.650964,-79.353041
4,M6A,North York,"Lawrence Manor , Lawrence Heights",43.723265,-79.451211
5,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.661790,-79.389390
6,M8A,Not assigned,M9A,43.648690,-79.385440
7,M1B,Scarborough,"Malvern , Rouge",43.808626,-79.189913
8,M2B,Not assigned,M3B,43.648690,-79.385440
9,M4B,East York,"Parkview Hill , Woodbine Gardens",43.707193,-79.311529


## Define Foursquare Credential and Versions

In [18]:
CLIENT_ID = 'SMW0KBR1QLGVOEBX3HFOQFVCABUB5HXWFTUZZCZMHFD5PVTW' # your Foursquare ID
CLIENT_SECRET = '3KAMZFVCZ15XQ04SVTKVDMVTEFQJASZNQRSVTUUNUZHX4PDZ' # your Foursquare Secret
VERSION = '20200406' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: SMW0KBR1QLGVOEBX3HFOQFVCABUB5HXWFTUZZCZMHFD5PVTW
CLIENT_SECRET:3KAMZFVCZ15XQ04SVTKVDMVTEFQJASZNQRSVTUUNUZHX4PDZ


In [19]:
neighbors.loc[4, 'Neighborhood'].split(',')[0]

'Lawrence Manor '

In [20]:
neighborhood_latitude = neighbors.loc[4, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = neighbors.loc[4, 'Longitude'] # neighborhood longitude value

neighborhood_name = neighbors.loc[4, 'Neighborhood'].split(',')[0] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Lawrence Manor  are 43.723265465000054, -79.45121077799996.


### Now, let's get the top 100 venues that are in Marble Hill within a radius of 500 meters.

#### Creating Get Request URL

In [21]:
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    500, 
    100)

url

'https://api.foursquare.com/v2/venues/explore?&client_id=SMW0KBR1QLGVOEBX3HFOQFVCABUB5HXWFTUZZCZMHFD5PVTW&client_secret=3KAMZFVCZ15XQ04SVTKVDMVTEFQJASZNQRSVTUUNUZHX4PDZ&v=20200406&ll=43.723265465000054,-79.45121077799996&radius=500&limit=100'

In [22]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ea604451a4b0a74623d26a0'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Toronto',
  'headerFullLocation': 'Toronto',
  'headerLocationGranularity': 'city',
  'totalResults': 54,
  'suggestedBounds': {'ne': {'lat': 43.727765469500056,
    'lng': -79.44499563189912},
   'sw': {'lat': 43.71876546050005, 'lng': -79.4574259241008}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '50a856a5e4b0bfc9165ef55c',
       'name': 'Ted Baker London',
       'location': {'address': '3401 Dufferin St.',
        'crossStreet': 'in Yorkdale Shopping Centre',
        'lat': 43.72451893248142,
        'lng': -79.45270961233477,
        'labeledLatLngs': [{'label

Defining Function for extracting categories:

In [23]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now, we will clean json data and convert it into dataframe:

In [24]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Ted Baker London,Clothing Store,43.724519,-79.452710
1,Yorkdale Shopping Centre,Shopping Mall,43.725482,-79.452736
2,Nordstrom,Clothing Store,43.726054,-79.449443
3,The Lego Store,Toy / Game Store,43.725146,-79.452974
4,John Varvatos,Men's Store,43.724428,-79.453689


In [25]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

54 venues were returned by Foursquare.


# Exploring All Neighborhoods of Toronto

The following function fetches venues from foursquare api:

In [26]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            500, 
            100)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [27]:
toronto_venues = getNearbyVenues(names=neighbors['Neighborhood'],
                                   latitudes=neighbors['Latitude'],
                                   longitudes=neighbors['Longitude']
                                  )

M2A
Parkwoods
Victoria Village
Regent Park , Harbourfront
Lawrence Manor , Lawrence Heights
Queen's Park , Ontario Provincial Government
M9A
Malvern , Rouge
M3B
Parkview Hill , Woodbine Gardens
Garden District, Ryerson
Glencairn
M8B
West Deane Park , Princess Gardens , Martin Grove , Islington , Cloverdale
Rouge Hill , Port Union , Highland Creek
M3C
Woodbine Heights
St. James Town
Humewood-Cedarvale
M8C
Eringate , Bloordale Gardens , Old Burnhamthorpe , Markland Wood
Guildwood , Morningside , West Hill
M3E
The Beaches
Berczy Park
Caledonia-Fairbanks
M8E
M1G
M3G
Leaside
Central Bay Street
Christie
M8G
M1H
Hillcrest Village
Bathurst Manor , Wilson Heights , Downsview North
Thorncliffe Park
Richmond , Adelaide , King
Dufferin , Dovercourt Village
M8H
M1J
Fairview , Henry Farm , Oriole
Northwood Park , York University
East Toronto
Harbourfront East , Union Station , Toronto Islands
Little Portugal , Trinity
M8J
M1K
Bayview Village
Downsview
The Danforth West , Riverdale
Toronto Dominion C

In [28]:
print(toronto_venues.shape)
toronto_venues.head()

(6614, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M2A,43.64869,-79.38544,Soho House Toronto,43.648734,-79.386541,Speakeasy
1,M2A,43.64869,-79.38544,Four Seasons Centre for the Performing Arts,43.650592,-79.385806,Concert Hall
2,M2A,43.64869,-79.38544,Rosalinda,43.650252,-79.385156,Vegetarian / Vegan Restaurant
3,M2A,43.64869,-79.38544,Shangri-La Toronto,43.649129,-79.386557,Hotel
4,M2A,43.64869,-79.38544,Cafe Landwer,43.648753,-79.385367,Café


Checking how many venues are returned for each neighborhood:

In [29]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,5,5,5,5,5,5
"Alderwood , Long Branch",8,8,8,8,8,8
"Bathurst Manor , Wilson Heights , Downsview North",20,20,20,20,20,20
Bayview Village,3,3,3,3,3,3
"Bedford Park , Lawrence Manor East",20,20,20,20,20,20
...,...,...,...,...,...,...
Willowdale,58,58,58,58,58,58
"Willowdale , Newtonbrook",2,2,2,2,2,2
Woodbine Heights,12,12,12,12,12,12


Let's find out how many unique categories can be curated from all the returned venues:

In [31]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 256 uniques categories.


# Analyzing Each Neighborhood

In [37]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 
toronto_onehot.head()

,ATM,Accessories Store,Airport,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [38]:
toronto_onehot.shape

(6614, 256)

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category:

In [39]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,ATM,Accessories Store,Airport,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Agincourt,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.0
1,"Alderwood , Long Branch",0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.0
2,"Bathurst Manor , Wilson Heights , Downsview North",0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.05,0.000000,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.333333,0.0,0.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.0
4,"Bedford Park , Lawrence Manor East",0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124,Willowdale,0.0,0.0,0.0,0.017241,0.0,0.0,0.0,0.017241,0.0,...,0.000000,0.0,0.0,0.0,0.00,0.017241,0.0,0.0,0.0,0.0
125,"Willowdale , Newtonbrook",0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.0
126,Woodbine Heights,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.083333,0.0,...,0.000000,0.0,0.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.0
127,"York Mills , Silver Hills",0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.0


In [41]:
toronto_grouped.shape

(129, 256)

Let's print each neighborhood along with the top 5 most common venues

In [42]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
              venue  freq
0       Supermarket   0.2
1    Breakfast Spot   0.2
2   Badminton Court   0.2
3  Sushi Restaurant   0.2
4      Skating Rink   0.2


----Alderwood , Long Branch----
               venue  freq
0  Convenience Store  0.12
1        Pizza Place  0.12
2        Gas Station  0.12
3                Gym  0.12
4        Coffee Shop  0.12


----Bathurst Manor , Wilson Heights , Downsview North----
               venue  freq
0               Bank  0.10
1        Coffee Shop  0.10
2     Ice Cream Shop  0.05
3              Diner  0.05
4  Convenience Store  0.05


----Bayview Village----
                             venue  freq
0       Construction & Landscaping  0.33
1                      Flower Shop  0.33
2                            Trail  0.33
3                  Organic Grocery  0.00
4  Molecular Gastronomy Restaurant  0.00


----Bedford Park , Lawrence Manor East----
                venue  freq
0      Sandwich Place  0.10
1         Coffee Shop  0.10
2  Ital

                 venue  freq
0          Coffee Shop  0.07
1                Hotel  0.05
2                 Café  0.04
3  Japanese Restaurant  0.04
4  American Restaurant  0.03


----M1K----
                 venue  freq
0          Coffee Shop  0.07
1                Hotel  0.05
2                 Café  0.04
3  Japanese Restaurant  0.04
4  American Restaurant  0.03


----M1L----
                 venue  freq
0          Coffee Shop  0.07
1                Hotel  0.05
2                 Café  0.04
3  Japanese Restaurant  0.04
4  American Restaurant  0.03


----M1T----
                 venue  freq
0          Coffee Shop  0.07
1                Hotel  0.05
2                 Café  0.04
3  Japanese Restaurant  0.04
4  American Restaurant  0.03


----M1Y----
                 venue  freq
0          Coffee Shop  0.07
1                Hotel  0.05
2                 Café  0.04
3  Japanese Restaurant  0.04
4  American Restaurant  0.03


----M1Z----
                 venue  freq
0          Coffee Shop  0.07
1 

                             venue  freq
0                            Hotel   1.0
1                  Organic Grocery   0.0
2       Modern European Restaurant   0.0
3  Molecular Gastronomy Restaurant   0.0
4              Monument / Landmark   0.0


----Northwood Park , York University----
                    venue  freq
0  Furniture / Home Store  0.14
1             Coffee Shop  0.07
2    Caribbean Restaurant  0.07
3           Metro Station  0.07
4        Sushi Restaurant  0.07


----Old Mill South , King's Mill Park , Sunnylea , Humber Bay , Mimico NE , The Queensway East , Royal York South East , Kingsway Park South East----
                     venue  freq
0             Home Service   0.4
1                     Bank   0.2
2                     Park   0.2
3              Flower Shop   0.2
4  New American Restaurant   0.0


----Parkdale , Roncesvalles----
                         venue  freq
0                  Coffee Shop  0.09
1                   Restaurant  0.05
2  Eastern European Rest

Now, We will convert it into dataframe. First, let's write a function to sort the venues in descending order.

In [43]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [143]:
num_top_venues = 3

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,Agincourt,Breakfast Spot,Skating Rink,Badminton Court
1,"Alderwood , Long Branch",Convenience Store,Sandwich Place,Pizza Place
2,"Bathurst Manor , Wilson Heights , Downsview North",Bank,Coffee Shop,Bridal Shop
3,Bayview Village,Construction & Landscaping,Flower Shop,Trail
4,"Bedford Park , Lawrence Manor East",Sandwich Place,Italian Restaurant,Coffee Shop


# Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [144]:
# set number of clusters
kclusters = 4

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [145]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = neighbors

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,M1A,Not assigned,M2A,43.648690,-79.385440,0.0,Coffee Shop,Hotel,Japanese Restaurant
1,M3A,North York,Parkwoods,43.752935,-79.335641,2.0,Food & Drink Shop,Park,Donut Shop
2,M4A,North York,Victoria Village,43.728102,-79.311890,0.0,Portuguese Restaurant,Intersection,Pizza Place
3,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.650964,-79.353041,0.0,Pub,Coffee Shop,Café
4,M6A,North York,"Lawrence Manor , Lawrence Heights",43.723265,-79.451211,0.0,Clothing Store,Cosmetics Shop,Toy / Game Store


In [146]:
toronto_merged[toronto_merged['Cluster Labels'] == 1]

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
104,M1V,Scarborough,"Milliken , Agincourt North , Steeles East , L'...",43.817998,-79.280887,1.0,Pharmacy,Yoga Studio,Donut Shop


In [147]:
toronto_merged[toronto_merged['Cluster Labels'] == 0]

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,M1A,Not assigned,M2A,43.648690,-79.385440,0.0,Coffee Shop,Hotel,Japanese Restaurant
2,M4A,North York,Victoria Village,43.728102,-79.311890,0.0,Portuguese Restaurant,Intersection,Pizza Place
3,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.650964,-79.353041,0.0,Pub,Coffee Shop,Café
4,M6A,North York,"Lawrence Manor , Lawrence Heights",43.723265,-79.451211,0.0,Clothing Store,Cosmetics Shop,Toy / Game Store
5,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.661790,-79.389390,0.0,Coffee Shop,Sushi Restaurant,Café
...,...,...,...,...,...,...,...,...,...
130,M9Y,Not assigned,M1Z,43.648690,-79.385440,0.0,Coffee Shop,Hotel,Japanese Restaurant
131,M2Z,Not assigned,M3Z,43.648690,-79.385440,0.0,Coffee Shop,Hotel,Japanese Restaurant
132,M4Z,Not assigned,M5Z,43.648690,-79.385440,0.0,Coffee Shop,Hotel,Japanese Restaurant
133,M6Z,Not assigned,M7Z,43.648690,-79.385440,0.0,Coffee Shop,Hotel,Japanese Restaurant


In [148]:
toronto_merged.dropna(inplace=True)

Finally, Let's visualize the clusters:

In [149]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    cluster = int(cluster)
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Examine Clusters

In [150]:
cluster1 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
cluster1

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,Not assigned,0.0,Coffee Shop,Hotel,Japanese Restaurant
2,North York,0.0,Portuguese Restaurant,Intersection,Pizza Place
3,Downtown Toronto,0.0,Pub,Coffee Shop,Café
4,North York,0.0,Clothing Store,Cosmetics Shop,Toy / Game Store
5,Downtown Toronto,0.0,Coffee Shop,Sushi Restaurant,Café
...,...,...,...,...,...
130,Not assigned,0.0,Coffee Shop,Hotel,Japanese Restaurant
131,Not assigned,0.0,Coffee Shop,Hotel,Japanese Restaurant
132,Not assigned,0.0,Coffee Shop,Hotel,Japanese Restaurant
133,Not assigned,0.0,Coffee Shop,Hotel,Japanese Restaurant


In [151]:
cluster2 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
cluster2

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
104,Scarborough,1.0,Pharmacy,Yoga Studio,Donut Shop


In [152]:
cluster3 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
cluster3

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
1,North York,2.0,Food & Drink Shop,Park,Donut Shop
18,York,2.0,Hockey Arena,Field,Park
25,York,2.0,Park,Bakery,Gym
34,North York,2.0,Residential Building (Apartment / Condo),Park,Dog Run
44,Downtown Toronto,2.0,Farm,Theme Park,Park
50,East Toronto,2.0,Discount Store,Bus Line,Grocery Store
59,North York,2.0,Bakery,Park,Yoga Studio
69,North York,2.0,Playground,Park,Yoga Studio
79,North York,2.0,Convenience Store,Park,Bank
82,West Toronto,2.0,Metro Station,Residential Building (Apartment / Condo),Gas Station


In [153]:
cluster4 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
cluster4

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
85,Scarborough,3.0,Auto Garage,Yoga Studio,Donut Shop


Well, We can conclude from above that neighborhoods of Toronto can be clustered into atleast 2 groups.